# 检查eol系统数据

In [ ]:

import pandas as pd
import numpy as np


from sqlalchemy import create_engine


ck_properties = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "5NsWWQeJ",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_properties_dev = {  # mysql 开发环境配置
    "host": "10.179.98.13",
    "port": "3306",
    "url": "jdbc:mysql://10.179.98.13:3306/ck",
    "user": "root",
    "password": "DMP_test135#",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

ck_engine = create_engine(
    f"clickhouse://{ck_properties['user']}:{ck_properties['password']}@{ck_properties['host']}:{ck_properties['port']}/{ck_properties['database']}"
)

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)

mysql_engine_dev = create_engine(
    f"mysql+pymysql://{mysql_properties_dev['user']}:{mysql_properties_dev['password']}" +
    f"@{mysql_properties_dev['host']}:{mysql_properties_dev['port']}/{mysql_properties_dev['database']}?charset=utf8"
)
                 

In [2]:
t_df = pd.read_sql(
    sql=f"select * from sdecdmp.etl_data where clt_date = '2023-01-01' limit 50",
    con=ck_engine
)
t_df.head()

,vin,ein,platname,data_source,data_type,clt_timestamp,clt_time,clt_date,hos_series,hos_mat_id,...,total_run_time,total_distance,total_fuel,PFltRgn_lSnceRgn,InjCrv_qPoI3Set,InjCtl_qCurr,Driving_regeneration,rgn_interval,Parking_regeneration,PFltRgn_stDem
0,LSC00922JS9300203,D9222002512,大通,snat,gc,1672551881,2023-01-01 05:44:41,2023-01-01,D系列,DBLF4833,...,1135.25,0.0,15784.0,0.0,0.0,22.47,0.0,2.0,0.0,None
1,LSC00922JS9300203,D9222002512,大通,snat,gc,1672551882,2023-01-01 05:44:42,2023-01-01,D系列,DBLF4833,...,1135.25,0.0,15784.0,0.0,0.0,20.26,0.0,2.0,0.0,None
2,LSC00922JS9300203,D9222002512,大通,snat,gc,1672551883,2023-01-01 05:44:43,2023-01-01,D系列,DBLF4833,...,1135.25,0.0,15784.0,0.0,0.0,22.56,0.0,2.0,0.0,None
3,LSC00922JS9300203,D9222002512,大通,snat,gc,1672552178,2023-01-01 05:49:38,2023-01-01,D系列,DBLF4833,...,1135.40,0.0,15784.0,0.0,0.0,15.93,0.0,2.0,0.0,None
4,LSC00922JS9300203,D9222002512,大通,snat,gc,1672552179,2023-01-01 05:49:39,2023-01-01,D系列,DBLF4833,...,1135.40,0.0,15784.0,0.0,0.0,21.97,0.0,2.0,0.0,None


In [4]:
# 读取NOTCCPPARAINFO.csv
pid_df = pd.read_sql(
    sql=f"select * from ods.eol_notccpparainfo",
    con=mysql_engine_prod
)
pid_df.head()

,ID,PID,ECUName,AcqstEnble,GetMode,UploadEnable,Uploadmode,Uploadtime,Channel,Style,...,dtcmaxnum,curdtcmark,Bz1,Bz2,pkgname,maxval,minval,active,group_code,signal_type
0,25077,C81_U010100,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0101,,,Y,7001,signed_int16
1,25078,C81_U010300,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0103,,,Y,7001,signed_int16
2,25079,C81_U010400,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0104,,,Y,7001,unsigned_int
3,25080,C81_U010500,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0105,,,Y,7001,signed_int16
4,25081,C81_U010600,C81,1,2,1,1,1,1,1,...,0,0,,Active,DID0106,,,Y,7001,signed_int16


In [5]:
model_df = pd.read_sql(
    sql=f"select * from ods.eol_modelnfo",
    con=mysql_engine_prod
)
model_df

,ID,ModelName,ModelStyle,ECUName,EditDate,Xgrgh,Xgr,Paralist,Bz1,VERSIONNUM,ACTIVE,NewModelStyle,modellist,PID
0,1,SCR效率分析_R_C81,1,C81,2022-09-16 10:16:00,20166,蔡乾睿,实时喷油量;上游氮氧浓度;车速;尿素喷射量;SCR上游排气温度;下游氮氧浓度;废气质量流量;...,None,1,1,1,None,SAE_J516099;SAE_S003216;SAE_J000084;SAE_J00659...
1,2,SCR效率分析_DC1W,1,DC1W,2022-09-16 09:43:00,20166,蔡乾睿,DPF出口温度;实时喷油量;发动机转速;上游氮氧浓度;车速;尿素喷射量;下游氮氧浓度;废气质...,None,1,1,1,None,SAE_A003241;SAE_J516099;SAE_J000190;SAE_S00321...
2,3,SCR效率分析_MD1,1,MD1,2022-09-16 11:27:00,20166,蔡乾睿,SCR上游排气温度;发动机转速;上游氮氧浓度;车速;下游氮氧浓度;废气质量流量;实时喷油量;,None,1,1,1,None,SAE_J004360;SAE_J000190;SAE_J003216;SAE_J00008...
3,4,单车油耗_R_C81,1,C81,2022-09-16 13:37:00,20166,蔡乾睿,实时喷油量;车速;总行驶里程;总油耗;发动机转速;油门踏板;发动机总运行时间;,None,1,1,1,None,SAE_J516099;SAE_J000084;SAE_J000245;SAE_J00025...
4,5,单车油耗_DC1W,1,DC1W,2022-09-16 13:53:00,20166,蔡乾睿,实时喷油量;发动机转速;车速;总行驶里程;发动机总运行时间;总油耗;PTO开关;,None,1,1,1,None,SAE_J516099;SAE_J000190;SAE_J000084;SAE_J00024...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,61,喷嘴堵塞_MD1,1,MD1,2022-09-21 17:46:00,20283,陈玉倩,发动机转速;尿素泵压力;尿素浓度;尿素喷嘴占空比;尿素泵占空比;SCR主状态;SCR子状态,None,1,1,1,None,SAE_J000190;MD1_U122300;SAE_J003516;MD1_U12060...
61,62,跛行倒计时_C81,1,C81,2022-09-21 17:46:00,20283,陈玉倩,限扭倒计时;限速倒计时;,None,1,1,1,None,SAE_RFF2207;SAE_RFF2208
62,63,怠速过长_DC1W,1,DC1W,2022-09-21 17:46:00,20283,陈玉倩,空档开关;PTO控制状态;车速;油门踏板;发动机燃料流量;发动机运行模式,None,1,1,1,None,DCW_U010202;SAE_J000976;SAE_J000084;SAE_J00009...
63,64,怠速过长_MD1,1,MD1,2022-09-21 17:46:00,20283,陈玉倩,PTO控制状态;车速;油门踏板;发动机燃料流量;发动机运行模式,None,1,1,1,None,SAE_J000976;SAE_J000084;SAE_J000091;SAE_J00018...


In [6]:
from pprint import pprint

# 获取model信息表中文名与pid对应关系
pid_desc_dict = {}
for _, row in model_df.iterrows():
    para_list = row['Paralist'].split(';')
    pid_list = row['PID'].split(';')
    for desc, pid in zip(para_list, pid_list):
        if pid == '': continue
            
        if pid in pid_desc_dict:
            pid_desc_dict[pid].append(desc)
        else:
            pid_desc_dict[pid] = [desc]
        
        if pid == 'MD1_U113100' and desc == '实时喷油量':
            print(row)

# for pid in pid_desc_dict:
#     pid_desc_dict[pid] = set(pid_desc_dict[pid])

# pprint(pid_desc_dict)

In [7]:
from collections import Counter
for pid in pid_desc_dict:
    cnter = Counter(pid_desc_dict[pid])
    if len(cnter) > 1:
        print(pid, cnter)
# pprint(pid_desc_dict)

C81_U011D00 Counter({'蓄电池电压': 1, '蓄电池电压(mV)': 1})
SAE_J000168 Counter({'蓄电池电压': 3, '蓄电池电压(V)': 1})
null Counter({'油门踏板': 1, '进气歧管压力': 1, '大气压力': 1})
